In [1]:
library(DSS)

Loading required package: Biobase

Loading required package: BiocGenerics

Loading required package: generics


Attaching package: ‘generics’


The following objects are masked from ‘package:base’:

    as.difftime, as.factor, as.ordered, intersect, is.element, setdiff,
    setequal, union



Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    Filter, Find, Map, Position, Reduce, anyDuplicated, aperm, append,
    as.data.frame, basename, cbind, colnames, dirname, do.call,
    duplicated, eval, evalq, get, grep, grepl, is.unsorted, lapply,
    mapply, match, mget, order, paste, pmax, pmax.int, pmin, pmin.int,
    rank, rbind, rownames, sapply, saveRDS, table, tapply, unique,
    unsplit, which.max, which.min


Welcome to Bioconductor

    Vignettes contain introductory material; view with
    'browseVignettes()'. To cite Bioconductor, see
    'citation("Bioba

In [2]:
library(BiocParallel)

In [18]:
convert_pacbio_count <- function(file, outfile, min_coverage = 5, verbose = FALSE) {
  if (verbose) message("Reading PacBio file...")
  
  df <- fread(file, header=TRUE)
  setnames(df, gsub("^#", "", colnames(df))) # rename '#chrom' → 'chrom'
  
  required_cols <- c("chrom","begin","end","cov","mod_count")
  if (!all(required_cols %in% colnames(df))) {
    stop("PacBio file must contain columns: ", paste(required_cols, collapse=", "))
  }
  
  if (verbose) {
    message("Initial sites: ", nrow(df))
    message("Mean coverage: ", round(mean(df$cov), 2))
  }
  
  dss_df <- df %>%
    mutate(chr = chrom,
           pos = begin + 1,       # convert 0-based to 1-based
           N   = cov,
           X   = mod_count) %>%
    filter(N >= min_coverage) %>%  # Apply coverage filter
    select(chr, pos, N, X)
  
  if (verbose) {
    message("Sites after filtering (coverage >= ", min_coverage, "): ", nrow(dss_df))
    message("Mean methylation rate: ", round(mean(dss_df$X / dss_df$N), 3))
  }
  
  write.table(dss_df, file=outfile, sep="\t", quote=FALSE,
              row.names=FALSE, col.names=TRUE)
  
  return(nrow(dss_df))
}

In [19]:
hg002_dss <- convert_pacbio_count("data/HG002.pacbio.chr22.count.combined.bed.gz",
                                  "data/HG002.pacbio.chr22.count.dss.bed", verbose=TRUE)

Reading PacBio file...

Initial sites: 597011

Mean coverage: 30.34

Sites after filtering (coverage >= 5): 597011

Mean methylation rate: 0.671



In [23]:
st001Lung_hap1_dss <- convert_pacbio_count("data/st001.lung.pacbio.chr22.count.hap1.bed.gz",
                                  "data/st001.lung.pacbio.chr22.hap1.count.dss.bed", verbose=TRUE)

st001Lung_hap2_dss <- convert_pacbio_count("data/st001.lung.pacbio.chr22.count.hap2.bed.gz",
                                  "data/st001.lung.pacbio.chr22.hap2.count.dss.bed", verbose=TRUE)

Reading PacBio file...

Initial sites: 552273

Mean coverage: 12.86

Sites after filtering (coverage >= 5): 552273

Mean methylation rate: 0.69

Reading PacBio file...

Initial sites: 548166

Mean coverage: 13.02

Sites after filtering (coverage >= 5): 548166

Mean methylation rate: 0.692



In [24]:
sample_files <- c("data//HG002.pacbio.chr22.count.dss.bed", "data/st001.lung.pacbio.chr22.hap2.count.dss.bed",
                 "data/st001.lung.pacbio.chr22.hap1.count.dss.bed")

In [25]:
sample_names <- c("HG002_pb","st001Lung_hap1","st001Lung_hap2")

In [16]:
load_bs_data <- function(sample_files, sample_names, coverage_threshold = 5) {
  
  cat("Loading bisulfite sequencing data files...\n")
  
  # Validate inputs
  if(length(sample_files) != length(sample_names)) {
    stop("Number of sample files must match number of sample names")
  }
  
  # Load all data files
  dat_list <- list()
  for(i in 1:length(sample_files)) {
    cat(paste("Loading", sample_files[i], "\n"))
    
    if(!file.exists(sample_files[i])) {
      stop(paste("File not found:", sample_files[i]))
    }
    
    dat_list[[i]] <- read.table(sample_files[i], header=TRUE, stringsAsFactors=FALSE)
    
    # Check data format
    required_cols <- c("chr", "pos", "N", "X")
    if(!all(required_cols %in% colnames(dat_list[[i]]))) {
      stop(paste("File", sample_files[i], "missing required columns:", 
                 paste(required_cols[!required_cols %in% colnames(dat_list[[i]])], collapse=", ")))
    }
    
    # Basic data validation
    cat(paste("  Rows:", nrow(dat_list[[i]]), "\n"))
    cat(paste("  Chromosomes:", length(unique(dat_list[[i]]$chr)), "\n"))
    cat(paste("  Mean coverage:", round(mean(dat_list[[i]]$N, na.rm=TRUE), 2), "\n"))
  }
  
  # Create BSseq object
  cat("Creating BSseq object...\n")
  BSobj <- makeBSseqData(dat_list, sampleNames = sample_names)
  
  # Filter out CpG sites with low coverage
  cat(paste("Filtering sites with coverage <", coverage_threshold, "\n"))
  initial_sites <- nrow(BSobj)
  BSobj <- BSobj[rowSums(getCoverage(BSobj, type="Cov")) >= coverage_threshold]
  final_sites <- nrow(BSobj)
  
  cat(paste("CpG sites before filtering:", initial_sites, "\n"))
  cat(paste("CpG sites after filtering:", final_sites, "\n"))
  cat(paste("Sites removed:", initial_sites - final_sites, "\n"))
  
  cat("BSseq object created successfully!\n")
  print(BSobj)
  
  return(BSobj)
}

In [26]:
BSobj_raw <- load_bs_data(sample_files, sample_names, coverage_threshold = 5)


Loading bisulfite sequencing data files...
Loading data//HG002.pacbio.chr22.count.dss.bed 
  Rows: 597011 
  Chromosomes: 1 
  Mean coverage: 30.34 
Loading data/st001.lung.pacbio.chr22.hap2.count.dss.bed 
  Rows: 548166 
  Chromosomes: 1 
  Mean coverage: 13.02 
Loading data/st001.lung.pacbio.chr22.hap1.count.dss.bed 
  Rows: 552273 
  Chromosomes: 1 
  Mean coverage: 12.86 
Creating BSseq object...
Filtering sites with coverage < 5 
CpG sites before filtering: 611707 
CpG sites after filtering: 611707 
Sites removed: 0 
BSseq object created successfully!
An object of type 'BSseq' with
  611707 methylation loci
  3 samples
has not been smoothed
All assays are in-memory


In [21]:
smooth_bsobj <- function(bsobj, cores = 2, save_output = TRUE, output_name = NULL) {
  
  if(class(bsobj)[1] != "BSseq") {
    stop("Input must be a BSseq object")
  }
  
  cat(paste("Smoothing", ncol(bsobj), "samples with", cores, "cores...\n"))
  start_time <- Sys.time()
  
  # Apply smoothing
  bsobj_smooth <- BSmooth(bsobj, BPPARAM = MulticoreParam(workers = cores))
  
  end_time <- Sys.time()
  duration <- round(difftime(end_time, start_time, units="mins"), 2)
  cat(paste("Completed in", duration, "minutes\n"))
  
  # Save if requested
  if(save_output) {
    if(is.null(output_name)) {
      output_name <- paste0("BSobj_smoothed_", format(Sys.time(), "%Y%m%d_%H%M%S"), ".RData")
    }
    save(bsobj_smooth, file = output_name)
    cat(paste("Saved to", output_name, "\n"))
  }
  
  return(bsobj_smooth)
}

In [28]:
smoothed_HG002_pb <- smooth_bsobj(BSobj_raw, cores=4)

Smoothing 3 samples with 4 cores...


Warning message in parallel::mccollect(wait = FALSE, timeout = 1):
“1 parallel job did not deliver a result”


ERROR: Error in reducer$value.cache[[as.character(idx)]] <- values: wrong args for environment subassignment
